In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

REPO_DIR = '/content/emergence-lab'
GITHUB_USERNAME = 'imashishkh21'

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull origin main

os.chdir(REPO_DIR)
!pip install -e ".[dev]" -q

import jax
print(f"JAX devices: {jax.devices()}")


In [ ]:
from src.configs import Config

config = Config()
config.env.grid_size = 20
config.env.num_agents = 8
config.env.num_food = 10
config.evolution.enabled = True
config.evolution.max_agents = 32
config.evolution.starting_energy = 200
config.evolution.max_energy = 300  # raised from 200 — allows energy accumulation for reproduction
config.evolution.food_energy = 100
config.evolution.reproduce_threshold = 120
config.evolution.reproduce_cost = 80
config.evolution.mutation_std = 0.01
config.train.total_steps = 1_000_000
config.train.num_envs = 32
config.train.num_steps = 128
config.log.wandb = False
config.log.save_interval = 0
# Field ON with default settings (no special pheromone tuning)
# Ch1 OFF for simplicity
config.field.channel_decay_rates = (0.05, 1.0, 0.0, 0.0)
# No patch scaling
config.nest.patch_scaling_enabled = False

print("Config ready")
print(f"max_energy: {config.evolution.max_energy}")
print(f"total_steps: {config.train.total_steps}")

In [ ]:
import time
from src.training.parallel_train import ParallelTrainer

checkpoint_dir = '/content/drive/MyDrive/emergence-lab/delivery_test_v4'
steps_per_iter = config.train.num_envs * config.train.num_steps * config.evolution.max_agents
num_iterations = max(1, config.train.total_steps // steps_per_iter)

print("steps_per_iter:", steps_per_iter)
print("num_iterations:", num_iterations)

trainer = ParallelTrainer(
    config=config,
    num_seeds=1,
    seed_ids=[0],
    checkpoint_dir=checkpoint_dir,
    master_seed=42,
)

t0 = time.time()
metrics = trainer.train(
    num_iterations=num_iterations,
    checkpoint_interval_minutes=30,
    resume=False,
    print_interval=1,
)
elapsed = time.time() - t0

print(f"Training finished in {elapsed/60:.1f} minutes")
print("Available metric keys:", sorted(metrics.keys()))

In [ ]:
import numpy as np

reward = float(np.mean(np.asarray(metrics.get('mean_reward', [0.0]), dtype=float)))
population = float(np.mean(np.asarray(metrics.get('population_size', [0.0]), dtype=float)))

pickups = float(np.mean(np.asarray(metrics.get('num_pickups', [0.0]), dtype=float)))
deliveries = float(np.mean(np.asarray(metrics.get('num_deliveries', [0.0]), dtype=float)))
delivery_rate = deliveries / pickups if pickups > 0 else 0.0

print("Final metrics")
print("------------")
print(f"mean_reward:     {reward:.4f}")
print(f"population_size: {population:.2f}")
print(f"num_pickups:     {pickups:.2f}")
print(f"num_deliveries:  {deliveries:.2f}")
print(f"delivery_rate:   {delivery_rate:.3f}")
print()

# Population verdict
pop_verdict = "STABLE" if population >= 8.0 else "DECLINING"
print(f"Population: {pop_verdict} (started at 8, now {population:.2f})")

# Delivery verdict
del_verdict = "PASS" if delivery_rate >= 0.3 else "FAIL"
print(f"Delivery:   {del_verdict} (rate {delivery_rate:.3f}, threshold 0.300)")

# Overall
overall = "GREEN LIGHT" if pop_verdict == "STABLE" and del_verdict == "PASS" else "NEEDS WORK"
print(f"\nOverall:    {overall}")